In [1]:
import numpy as np
import pandas as pd

# Load the datasets
fridge_data = pd.read_csv('My_data_fridge_EXP.csv')
power_data = pd.read_csv('averaged_power_data.csv')

# Convert TIME to a common format (if necessary)
fridge_data['TIME'] = pd.to_datetime(fridge_data['TIME'], format='%H:%M:%S').dt.time
power_data['TIME'] = pd.to_datetime(power_data['TIME'], format='%H:%M:%S').dt.time

# Merge the two datasets on the TIME column
merged_data = pd.merge(fridge_data, power_data, on='TIME')

# Set initial conditions
BES_capacity = 1.5  # Backup energy source capacity in kWh (fully charged at midnight)
BES_current = BES_capacity  # Start fully charged
L = 3.0  # Initial lower limit for temperature
U = 7.0  # Upper limit for temperature
delta = 0.1  # Rate at which L will be increased

# Function to determine power source for each row
def determine_power_source(row):
    global BES_current, L  # To track BES usage and lower limit

    res_available = row['GES(inKWh)']  # Renewable energy available
    fridge_power_demand = row['Power_final']  # Power demand from the averaged power data
    
    # Condition 1: If RES is available and sufficient to meet the demand
    if res_available >= fridge_power_demand:
        return "RES", fridge_power_demand
    
    # Condition 2: If RES is insufficient and just exhausted
    elif res_available < fridge_power_demand and res_available == 0:
        # Increase lower limit L toward U at the rate delta, if L < U
        if L < U:
            L += delta
        # Now check if BES can fulfill the demand
        if BES_current >= fridge_power_demand:
            BES_current -= fridge_power_demand  # Deduct from BES
            return "BES", fridge_power_demand
        else:
            return "Grid", fridge_power_demand

    # Condition 3: If RES is insufficient, use BES if available
    elif BES_current >= fridge_power_demand:
        BES_current -= fridge_power_demand  # Deduct from BES
        return "BES", fridge_power_demand
    
    # Condition 4: If both RES and BES are insufficient, use grid power
    else:
        return "Grid", fridge_power_demand

# Apply the function row by row to determine the power source
merged_data['Power_Source'], merged_data['Power_Used'] = zip(*merged_data.apply(determine_power_source, axis=1))

# Reset BES at midnight (assuming the TIME is sorted)
def reset_bes_at_midnight(row):
    global BES_current
    if row['TIME'] == pd.to_datetime('00:00:00', format='%H:%M:%S').time():
        BES_current = BES_capacity  # Reset BES to full capacity at midnight
    return BES_current

# Apply the BES reset at midnight
merged_data['BES_After_Reset'] = merged_data.apply(reset_bes_at_midnight, axis=1)

# Save the final data with power source selection
merged_data.to_csv('optimized_power_usage.csv', index=False)

print("Power optimization process complete! Results saved to 'optimized_power_usage.csv'.")



Power optimization process complete! Results saved to 'optimized_power_usage.csv'.


In [2]:
# Initialize BES and temperature limits
BES_capacity = 1.5  # Full capacity in kWh at midnight
BES_current = BES_capacity  # Start with full charge
L = 3.0  # Lower temperature limit
U = 7.0  # Upper temperature limit
delta = 0.1  # Rate to increase L towards U

# Function to determine power source for each row, following your specified order
def determine_power_source(row):
    global BES_current, L  # Track BES and temperature adjustments

    res_available = row['GES(inKWh)']  # Renewable energy available
    fridge_power_demand = row['Power_final']  # Power demand

    # Step 1: If RES is sufficient, use RES
    if res_available >= fridge_power_demand:
        return "RES", fridge_power_demand

    # Step 2: If RES is exhausted, adjust L towards U by delta
    elif res_available < fridge_power_demand and res_available == 0:
        # Increase L towards U
        if L < U:
            L += delta

        # After adjusting L, check BES
        if BES_current >= fridge_power_demand:
            BES_current -= fridge_power_demand
            return "BES", fridge_power_demand
        else:
            return "Grid", fridge_power_demand  # Fall back to grid if BES is not enough

    # Step 3: If RES is partially available, try BES if it can meet demand
    elif BES_current >= fridge_power_demand:
        BES_current -= fridge_power_demand
        return "BES", fridge_power_demand

    # Step 4: If RES and BES are insufficient, use grid power
    else:
        return "Grid", fridge_power_demand

# Apply the function to determine power source row by row
merged_data['Power_Source'], merged_data['Power_Used'] = zip(*merged_data.apply(determine_power_source, axis=1))

# Function to reset BES to full charge at midnight
def reset_bes_at_midnight(row):
    global BES_current
    if row['TIME'] == pd.to_datetime('00:00:00', format='%H:%M:%S').time():
        BES_current = BES_capacity  # Reset BES to full at midnight
    return BES_current

# Apply the reset function
merged_data['BES_After_Reset'] = merged_data.apply(reset_bes_at_midnight, axis=1)

# Save the final data with power source selections
merged_data.to_csv('optimized_power_usage.csv', index=False)

print("Power optimization process complete! Results saved to 'Optimized_power_usage.csv'.")


Power optimization process complete! Results saved to 'Optimized_power_usage.csv'.
